In [19]:
import torch
from transformers import pipeline
import numpy as np
import pandas as pd

2023-12-09 14:32:43.062106: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-09 14:32:43.062148: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-09 14:32:43.069019: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-09 14:32:44.966832: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-09 14:32:52.485804: W tensorflow/compiler/

In [20]:
sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [21]:
data = pd.read_csv('/home/andyliu/fa23-classes/anlp-figlang/data/test_enhanced/hi.csv')
data.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'ending1', 'ending2', 'startphrase',
       'labels', 'en_startphrase', 'vehicle', 'vehicle_characteristics',
       'characteristics_suffix', 'characteristics_suffix_final',
       'startphrase_enhanced', 'en_ending1', 'en_ending2'],
      dtype='object')

In [43]:
def convert_to_numeric(pipeline_obj):
    if pipeline_obj['label'] == 'NEGATIVE':
        return(1 - pipeline_obj['score'])
    else:
        return(pipeline_obj['score'])

def add_sentiment_label(data_path, sentiment_pipeline):
    data = pd.read_csv(data_path)
    startphrase = np.array([convert_to_numeric(x) for x in sentiment_pipeline(list(data['en_startphrase']))])
    ending1 = np.array([convert_to_numeric(x) for x in sentiment_pipeline(list(data['en_ending1']))])
    ending2 = np.array([convert_to_numeric(x) for x in sentiment_pipeline(list(data['en_ending2']))])
    labels = list((np.abs(startphrase - ending1) > np.abs(startphrase - ending2)).astype(int))
    data['sent_label'] = labels
    data.to_csv(data_path)

In [44]:
add_sentiment_label('/home/andyliu/fa23-classes/anlp-figlang/data/test_enhanced/hi.csv', sentiment_pipeline)

In [45]:
for lang in ['id', 'jv', 'kn', 'su', 'sw', 'yo']:
    add_sentiment_label(f'/home/andyliu/fa23-classes/anlp-figlang/data/test_enhanced/{lang}.csv', sentiment_pipeline)